In [2]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [3]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 390, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 390 (delta 89), reused 53 (delta 51), pack-reused 269
Receiving objects: 100% (390/390), 107.16 KiB | 870.00 KiB/s, done.
Resolving deltas: 100% (191/191), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 1.8 MB/s eta 0:00:00
***********************************************************************
Woo! Your instance has the right kind of GPU, a Tesla T4!
We will now install RAPIDS cuDF, cuML, and cuGraph via pip! 
Please stand by, should be quick...
***********************************************************************

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.5/502.5 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 GB 568.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
!python -m pip install "dask-ml"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.7/148.7 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.0/81.0 kB 5.5 MB/s eta 0:00:00


In [5]:
import pandas as pd
from sklearn.linear_model import SGDClassifier
from cuml.model_selection import train_test_split
from cuml.metrics import accuracy_score
import os
import cudf as cd
import dask_cudf as dc
from cuml.decomposition import pca
import numpy as np

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Reading the file from drive

In [7]:
path = '/content/drive/MyDrive/Colab Notebooks/Data Mining'
dir_list = os.listdir(path)
df2 = cd.DataFrame()
for i in dir_list:
    if df2.empty:
        df2 = cd.read_csv(path + '/' + i, sep='|', na_values='-')
    else:
        df2 = cd.concat([cd.read_csv(path + '/' + i, sep='|', na_values='-'), df2], axis=0)

###We have total of 2.5 crore rows from which I'm considering the rows between 50 lakhs to 1 crore, that is total of 50 lakhs,

###The reason behind this is at the time of Model training the GPU goes out of memory when doing hyperparameter tuning. Without hyper-parameter the model is training without going out of memory.

In [8]:
df2 = df2[5000001:10000001]

In [9]:
df2.shape

(5000000, 23)

In [10]:
df2.columns

Index(['ts', 'uid', 'id.orig_h', 'id.orig_p', 'id.resp_h', 'id.resp_p',
       'proto', 'service', 'duration', 'orig_bytes', 'resp_bytes',
       'conn_state', 'local_orig', 'local_resp', 'missed_bytes', 'history',
       'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes',
       'tunnel_parents', 'label', 'detailed-label'],
      dtype='object')

###Dropping the columns 'uid', 'local_orig', 'local_resp', 'tunnel_parents', 'detailed-label', 'id.orig_h', 'id.resp_h' to finalize the data for model training.

In [11]:
df = df2.drop(columns=['uid', 'local_orig', 'local_resp', 'tunnel_parents', 'detailed-label', 'id.orig_h', 'id.resp_h'])
del df2

##Applying One-hot Encoder on categorical columns protocol, connection_state, service.

##Applying Label-encoder on numercial column history.

In [12]:
df = cd.get_dummies(df, columns=['proto', 'conn_state', 'service'])
df['history'] = cd.factorize(df['history'])[0] + 1

/usr/local/lib/python3.10/dist-packages/cudf/core/reshape.py:708: FutureWarning: Default `dtype` value will be changed to 'bool' in a future release, please update `dtype='bool'` to adapt for future behavior.
  warnings.warn(


##Converting the duration, orig_bytes, resp_bytes to floating type from object.

In [13]:
df['duration'] = df['duration'].astype('float')
df['orig_bytes'] = df['orig_bytes'].astype('float')
df['resp_bytes'] = df['resp_bytes'].astype('float')

In [14]:
df.shape

(5000000, 29)

##Applying Label-Encoder on Label column

In [15]:
df['label'] = cd.factorize(df['label'])[0]

In [16]:
df = df.fillna(0)

##Finalizing the input and output column.

In [17]:
x = df.drop(columns=['label'])
y = df['label']

In [18]:
del df

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [20]:
del x
del y

##Applying  Multinomial Naive Bayes model in data

In [21]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV

In [22]:
param_grid = {

    'alpha': [0.01, 0.1],  # Smoothing parameter
    'fit_prior': [True, False],  # Whether to learn class prior probabilities
}

naive_bayes = MultinomialNB()
grid_search = GridSearchCV(estimator=naive_bayes, param_grid=param_grid, scoring='accuracy', cv=5)
grid_search.fit(X_train.to_numpy(), y_train.to_numpy())

GridSearchCV(cv=5, estimator=MultinomialNB(),
             param_grid={'alpha': [0.01, 0.1], 'fit_prior': [True, False]},
             scoring='accuracy')

In [23]:
y_pred = grid_search.predict(X_test.to_numpy())

##Printing the best hyper-parameters of accuracy

In [24]:
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

Best Hyperparameters: {'alpha': 0.01, 'fit_prior': True}


In [25]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(y_test.to_numpy(), y_pred)
precision = precision_score(y_pred, y_test.to_numpy(), average=None)
recall = recall_score(y_pred, y_test.to_numpy(),average=None)
f1 = f1_score(y_test.to_numpy(), y_pred, average=None)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)


Accuracy: 0.8879813333333333
Precision: [6.02986443e-01 5.87889477e-04 6.21621622e-01 9.99984169e-01
 7.84505422e-01 1.00000000e+00]
Recall: [0.98717333 0.125      0.88461538 0.99973    0.0549254  0.04455446]
F1-score: [0.74866959 0.00117028 0.73015873 0.99985707 0.10266307 0.08530806]


In [27]:
from sklearn.metrics import classification_report
report = classification_report(y_test.to_numpy(), y_pred)

print(report)

              precision    recall  f1-score   support

           0       0.99      0.60      0.75    412263
           1       0.12      0.00      0.00      1701
           2       0.88      0.62      0.73        37
           3       1.00      1.00      1.00   1073818
           4       0.05      0.78      0.10     12172
           5       0.04      1.00      0.09         9

    accuracy                           0.89   1500000
   macro avg       0.52      0.67      0.44   1500000
weighted avg       0.99      0.89      0.92   1500000

